# Perspective Streaming Data Example
In this example, we will use [csp](https://github.com/point72/csp) and [superstore](https://github.com/timkpaine/superstore) to build a streaming data graph, and pump data into `perspective` widgets.

`superstore` is a library with some simulated data sources, and `csp` is a stream processing library. The details of `csp` are not super important for the purpose of this talk, but here we construct a streaming DAG of data which will feed `perspective` at different intervals

In [ ]:
import csp
import pandas as pd
from datetime import timedelta
from ipywidgets import HBox, VBox
from perspective import PerspectiveWidget
from csp_nodes import *

In [ ]:
m = machines()
pd.DataFrame(m)

In [ ]:
u = usage(m[1])
u = usage(u)
u

In [ ]:
status(u)

In [ ]:
jobs(u)

In [ ]:
@csp.graph
def main_graph(
    machines_widget: PerspectiveWidget,
    usage_widget: PerspectiveWidget,
    status_widget: PerspectiveWidget,
    jobs_widget: PerspectiveWidget,
):
    # A randomly generated list of "machines"
    all_machines = machines()

    # construct three ticking nodes for usage, status, and jobs
    usage = machine_usage(all_machines, timedelta(seconds=1))
    status = machine_status(usage, timedelta(seconds=5))
    jobs = machine_jobs(all_machines, timedelta(seconds=5))

    # push all of our data to 4 separate perspective tables
    push_to_perspective(csp.const(all_machines), machines_widget)
    push_to_perspective(usage, usage_widget)
    push_to_perspective(status, status_widget)
    push_to_perspective(jobs, jobs_widget)

In [ ]:
# construct 4 separate perspective widgets. Each will have its own table internally
machines_widget = PerspectiveWidget(MACHINE_SCHEMA, index="machine_id", settings=False)
usage_widget = PerspectiveWidget(USAGE_SCHEMA, index="machine_id", settings=False)
status_widget = PerspectiveWidget(STATUS_SCHEMA, index="machine_id", sort=[["last_update", "desc"]], settings=False)
jobs_widget = PerspectiveWidget(JOBS_SCHEMA, sort=[["start_time", "desc"]], settings=False)

In [ ]:
# a little bit of layout with ipywidgets
VBox(children=[
    HBox(children=[machines_widget, usage_widget]),
    HBox(children=[status_widget, jobs_widget]),
])

In [ ]:
csp.run_on_thread(main_graph, machines_widget, usage_widget, status_widget, jobs_widget, realtime=True, daemon=True)

In [ ]:
status_widget.plugin = "X Bar"
status_widget.group_by = ["status"]
status_widget.columns = ["machine_id"]
status_widget.aggregates = {"status": "last"}